# Testing for UNION - using all spindles, training with all data - DREAMS
fistly define the UUID of the experiment to use

In [1]:
experimentId="77cdf395-3015-4c9e-a90b-a661d7715c16"

-------------------------------------------------------

In [2]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time
import sklearn.metrics as skm

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']
cognipath=datapath+"\\COGNITION"
dreamspath=datapath+"\\DREAMS"
masspath=datapath+"\\MASS"


## load mass

In [3]:
samplerate=200  #Should rethink this

In [4]:
signals, annotations, signalsMetadata = loadDREAMSSpindles(dreamspath)

SubjectId: 0001--------------
resampling from 100 to 200
SubjectId: 0003--------------
resampling from 50 to 200
SubjectId: 0006--------------
duration discrepancy, removing last 0.7 seconds


In [5]:
annotations.head()

,index,startTime,duration,channel,subjectId,labelerId,type,samplerate,stopTime,startInd,stopInd
0,0,282.24,0.72,C3-A1,0001,0001,spindle,200,282.96,56448,56592
1,1,311.72,1.54,C3-A1,0001,0001,spindle,200,313.26,62344,62652
2,2,340.28,0.72,C3-A1,0001,0001,spindle,200,341.00,68056,68200
3,3,366.83,0.65,C3-A1,0001,0001,spindle,200,367.48,73366,73496
4,4,373.74,0.50,C3-A1,0001,0001,spindle,200,374.24,74748,74848


In [6]:
signalsMetadata.head()

,filename,channel,subjectId,duration,samplerate,database
0,excerpt1.txt,C3-A1,0001,1800,200,DREAMS
1,excerpt2.txt,CZ-A1,0002,1800,200,DREAMS
2,excerpt3.txt,C3-A1,0003,1800,200,DREAMS
3,excerpt4.txt,CZ-A1,0004,1800,200,DREAMS
4,excerpt5.txt,CZ-A1,0005,1800,200,DREAMS


## Load experiment results

In [7]:
experimentModels, featureSelection = loadExperiment(experimentId,datapath)

In [8]:
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0007]","[0008, 0004]",0001,70cf34de-3c08-4b43-8ead-e9c6231c6447,0.017738
1,0001,E1,[0001],"[0003, 0005, 0006, 0007, 0008]","[0001, 0004]",0002,417d5651-7922-417e-b915-f19589634c82,0.017258
2,0001,E1,[0001],"[0001, 0002, 0004, 0005, 0006]","[0008, 0007]",0003,551ea23f-12d4-4e32-b1fb-4ebd5ba05946,0.025579
3,0001,E1,[0001],"[0001, 0002, 0003, 0005, 0008]","[0007, 0006]",0004,c7d39827-0b39-4afe-9a2c-6f4f40ef25db,0.019417
4,0001,E1,[0001],"[0001, 0003, 0006, 0007, 0008]","[0002, 0004]",0005,9dd371bc-8e64-4bfd-9989-63b703f890dd,0.017488
5,0001,E1,[0001],"[0001, 0002, 0004, 0005, 0008]","[0003, 0007]",0006,23a0ce88-30ed-4fdc-9ec5-50c6ea04fcfb,0.024179
6,0001,E1,[0001],"[0001, 0002, 0003, 0006, 0008]","[0004, 0005]",0007,369117df-e8c4-4beb-b47e-5fda47d9a8dc,0.020952
7,0001,E1,[0001],"[0001, 0002, 0004, 0005, 0006]","[0003, 0007]",0008,d59f29db-9691-415f-9934-421863857d6f,0.025579
8,0002,E2,[0002],"[0002, 0003, 0005, 0006, 0007]","[0008, 0004]",0001,eb49479e-c1aa-4443-a3ac-4d5e72cdc3d4,0.029889
9,0002,E2,[0002],"[0003, 0005, 0006, 0007, 0008]","[0001, 0004]",0002,4ba9282f-255e-4aff-bee3-1e14c48ba23a,0.024111


In [9]:
#we show the difference in class inbalance for the annotation criteria considered
experimentModels[['criteriumName','spindleTimeRate']].groupby('criteriumName').describe()

spindleTimeRate                                          \
                        count      mean       std       min       25%   
criteriumName                                                           
E1                        8.0  0.021024  0.003616  0.017258  0.017675   
E2                        8.0  0.032346  0.005787  0.024111  0.029222   
union                     8.0  0.044124  0.007071  0.034659  0.038348   

                                             
                    50%       75%       max  
criteriumName                                
E1             0.020184  0.024529  0.025579  
E2             0.031832  0.034860  0.040443  
union          0.044133  0.047949  0.053786

In [10]:
featureSelection

,window,characteristic,bandName,score
0,2,hjortActivity,beta2,69.0
1,1,hjortActivity,sigma,63.0
2,2,hjortActivity,beta1,42.0
3,2,hjortActivity,delta2,41.0
4,1.5,sigmaIndex,broadband,39.0
5,2,sigmaIndex,broadband,39.0
6,2,hjortActivity,theta,38.0
7,2,hjortMobility,beta1,37.0
8,2,hjortActivity,alpha,35.0
9,1,sigmaIndex,broadband,34.0


## Hyperparameter definition
this should come from a previous evaluation notebook

In [11]:
hyperClose=0.25
hyperDuration=0.3
hyperThres=0.5
hyperDepth=40

## Testing with E2 criterium
we test the optimal points for the prediction threshold and number of boost iterations in the different validation groups

In [12]:
experimentModels=experimentModels[experimentModels.criteriumName=='union'].reset_index(drop=True)
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0003,union,"[0001, 0002]","[0002, 0003, 0005, 0006, 0007]","[0008, 0004]",0001,fb98a323-465c-46e5-9345-3f7172b0ddc0,0.038041
1,0003,union,"[0001, 0002]","[0003, 0005, 0006, 0007, 0008]","[0001, 0004]",0002,5aad028b-c353-4073-a25a-4cc65a40d4f6,0.034659
2,0003,union,"[0001, 0002]","[0001, 0002, 0004, 0005, 0006]","[0008, 0007]",0003,c5a8d823-6cce-4551-b939-072d6243bb8f,0.053786
3,0003,union,"[0001, 0002]","[0001, 0002, 0003, 0005, 0008]","[0007, 0006]",0004,0d05fa46-525d-40aa-ab28-0e48cf1fd098,0.043449
4,0003,union,"[0001, 0002]","[0001, 0003, 0006, 0007, 0008]","[0002, 0004]",0005,d6448bf7-7c76-44a8-b3bd-f6797aa49e9c,0.038450
5,0003,union,"[0001, 0002]","[0001, 0002, 0004, 0005, 0008]","[0003, 0007]",0006,0f55c379-26b2-40cf-860c-26de621acaf7,0.046003
6,0003,union,"[0001, 0002]","[0001, 0002, 0003, 0006, 0008]","[0004, 0005]",0007,4bf1000f-a3ae-4f1f-b893-946ebd67e116,0.044817
7,0003,union,"[0001, 0002]","[0001, 0002, 0004, 0005, 0006]","[0003, 0007]",0008,b2158e6d-46b1-44f1-8684-f2f8d6b7b84c,0.053786


In [13]:
#threshold for the metric by event
masterIoU=0.2

In [14]:
def getIou(coordA,coordB):
    if (coordA[1]<coordB[0])|(coordB[1]<coordA[0]): #NOT INTERSECTION
        iou=0
    else:   #INTERSECTION
        inter=np.min((coordA[1]-coordB[0],coordB[1]-coordA[0]))
        union=np.max((coordA[1]-coordB[0],coordB[1]-coordA[0]))
        iou=inter/union
    return iou

def annotationPairToMetrics(annotations,detections,thresIoU=0.2):
    #get the coords
    gtCoords=zip(annotations.startInd,annotations.stopInd)
    outCoords=zip(detections.startInd,detections.stopInd)
    #calculate the iou vector
    iouVector=np.array(list(itt.starmap(getIou,itt.product(gtCoords,outCoords))))
    #reshape to a matrix
    iouMatrix=iouVector.reshape(len(annotations),len(detections))
    #binarize
    binarized=iouMatrix>thresIoU
    #calculateF1
    outF1=(np.sum(np.max(binarized,axis=0))+np.sum(np.max(binarized,axis=1)))/(len(annotations)+len(detections))
    recall=np.sum(np.max(binarized,axis=1))/len(annotations)
    precision=np.sum(np.max(binarized,axis=0))/len(detections)
    return outF1,recall,precision

In [15]:
rawF1s=[]
rawPrecisions=[]
rawRecalls=[]

f1s=[]
precisions=[]
recalls=[]

eventF1s=[]
eventPrecisions=[]
eventRecalls=[]

checks=[]

for ind,row in experimentModels.iterrows():
    print('*************************')
    print(str(ind+1)+' of '+str(len(experimentModels)) )
    #load model
    model=loadBooster(row.modelId,experimentId,datapath)

    testSubjectId=row.test
    #Define annotations criterium
    usedAnnotations=annotations[annotations.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
    #Load features and labels
    testFeatures=loadFeatureMatrix([testSubjectId],featureSelection,signalsMetadata,samplerate,datapath)
    testLabels=loadLabelsVector([testSubjectId],usedAnnotations,signalsMetadata,samplerate)

    #Predict
    testDMatrix=xgb.DMatrix(data=testFeatures)
    probabilities=model.predict(testDMatrix,iteration_range=(0,hyperDepth))
    rawLabels=probabilities>=hyperThres
    #Raw Metrics
    rawTp=np.sum(rawLabels*testLabels)
    rawFp=np.sum(rawLabels*(1-testLabels))
    rawTn=np.sum((1-rawLabels)*(1-testLabels))
    rawFn=np.sum((1-rawLabels)*testLabels)
    #Raw appends
    rawF1s.append(2*rawTp/(2*rawTp+rawFp+rawFn))
    rawPrecisions.append(rawTp/(rawTp+rawFp) )
    rawRecalls.append(rawTp/(rawTp+rawFn))
    #Process
    processedLabels=labelingProcess(rawLabels,hyperClose,hyperDuration,samplerate)
    #Processed metrics
    tp=np.sum(processedLabels*testLabels)
    fp=np.sum(processedLabels*(1-testLabels))
    tn=np.sum((1-processedLabels)*(1-testLabels))
    fn=np.sum((1-processedLabels)*testLabels)
    #Processed appends
    f1s.append(2*tp/(2*tp+fp+fn))
    precisions.append(tp/(tp+fp))
    recalls.append(tp/(tp+fn))

    #By-event metrics
    processedAnnotations=labelVectorToAnnotations(processedLabels,samplerate)
    gtAnnotations=labelVectorToAnnotations(testLabels,samplerate)   #<- or just filter the annotations
    f,r,p=annotationPairToMetrics(gtAnnotations,processedAnnotations)
    
    #calculate metrics
    eventF1s.append(f)
    eventPrecisions.append(p)
    eventRecalls.append(r)

#include metrics in the dataframe
experimentModels['rawF1']=rawF1s
experimentModels['rawPrecision']=rawPrecisions
experimentModels['rawRecall']=rawRecalls

experimentModels['f1']=f1s
experimentModels['precision']=precisions
experimentModels['recall']=recalls

experimentModels['eventF1']=eventF1s
experimentModels['eventPrecision']=eventPrecisions
experimentModels['eventRecall']=eventRecalls


*************************
1 of 8
*************************
2 of 8
*************************
3 of 8
*************************
4 of 8
*************************
5 of 8
*************************
6 of 8
*************************
7 of 8
*************************
8 of 8


In [16]:
dumpPickle('experimentModelsTest_union_temp.pkl',experimentModels)

In [17]:
experimentModels.columns

Index(['criteriumId', 'criteriumName', 'labelerIdList', 'train', 'val', 'test',
       'modelId', 'spindleTimeRate', 'rawF1', 'rawPrecision', 'rawRecall',
       'f1', 'precision', 'recall', 'eventF1', 'eventPrecision',
       'eventRecall'],
      dtype='object')

In [18]:
fig=px.scatter(experimentModels,x='rawF1',y='f1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['rawF1'], y=experimentModels['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [19]:
fig=px.scatter(experimentModels,x='rawF1',y='eventF1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['rawF1'], y=experimentModels['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [20]:
fig=px.scatter(experimentModels,x='eventF1',y='eventPrecision',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [21]:
fig=px.scatter(experimentModels,x='eventF1',y='eventRecall',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [22]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test').describe(percentiles=[0.5])

eventF1                                             eventPrecision  \
       count      mean std       min       50%       max          count   
test                                                                      
0001     1.0  0.710938 NaN  0.710938  0.710938  0.710938            1.0   
0002     1.0  0.601562 NaN  0.601562  0.601562  0.601562            1.0   
0003     1.0  0.732558 NaN  0.732558  0.732558  0.732558            1.0   
0004     1.0  0.471545 NaN  0.471545  0.471545  0.471545            1.0   
0005     1.0  0.684932 NaN  0.684932  0.684932  0.684932            1.0   
0006     1.0  0.684466 NaN  0.684466  0.684466  0.684466            1.0   
0007     1.0  0.225352 NaN  0.225352  0.225352  0.225352            1.0   
0008     1.0  0.300000 NaN  0.300000  0.300000  0.300000            1.0   

                                                 eventRecall                \
          mean std       min       50%       max       count      mean std   
test                                                                         
0001  0.745902 NaN  0.745902  0.745902  0.745902         1.0  0.679104 NaN   
0002  0.764706 NaN  0.764706  0.764706  0.764706         1.0  0.493506 NaN   
0003  0.761905 NaN  0.761905  0.761905  0.761905         1.0  0.704545 NaN   
0004  0.483333 NaN  0.483333  0.483333  0.483333         1.0  0.460317 NaN   
0005  0.646552 NaN  0.646552  0.646552  0.646552         1.0  0.728155 NaN   
0006  0.786517 NaN  0.786517  0.786517  0.786517         1.0  0.606838 NaN   
0007  0.150943 NaN  0.150943  0.150943  0.150943         1.0  0.444444 NaN   
0008  0.183962 NaN  0.183962  0.183962  0.183962         1.0  0.812500 NaN   

                                    
           min       50%       max  
test                                
0001  0.679104  0.679104  0.679104  
0002  0.493506  0.493506  0.493506  
0003  0.704545  0.704545  0.704545  
0004  0.460317  0.460317  0.460317  
0005  0.728155  0.728155  0.728155  
0006  0.606838  0.606838  0.606838  
0007  0.444444  0.444444  0.444444  
0008  0.812500  0.812500  0.812500

In [23]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean()

,test,eventF1,eventPrecision,eventRecall
0,0001,0.710938,0.745902,0.679104
1,0002,0.601562,0.764706,0.493506
2,0003,0.732558,0.761905,0.704545
3,0004,0.471545,0.483333,0.460317
4,0005,0.684932,0.646552,0.728155
5,0006,0.684466,0.786517,0.606838
6,0007,0.225352,0.150943,0.444444
7,0008,0.300000,0.183962,0.812500


In [24]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().mean()

test              1.250250e+27
eventF1           5.514191e-01
eventPrecision    5.654775e-01
eventRecall       6.161764e-01
dtype: float64

In [26]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().std()

C:\Users\Gurdiel\AppData\Local\Temp\ipykernel_16904\2931376690.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



eventF1           0.197198
eventPrecision    0.264683
eventRecall       0.137202
dtype: float64

In [27]:
auxPrecision=pd.DataFrame({
    'metric':'event precision',
    'value':experimentModels.eventPrecision,
    'event F1':experimentModels.eventF1
})

auxRecall=pd.DataFrame({
    'metric':'event recall',
    'value':experimentModels.eventRecall,
    'event F1':experimentModels.eventF1
})
visualTradeoff=pd.concat((auxPrecision,auxRecall))

In [28]:
fig=px.scatter(visualTradeoff,x='event F1',y='value',color='metric', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

### remove outliers

In [29]:
np.setdiff1d(experimentModels.test,['0007','0008'])

array(['0001', '0002', '0003', '0004', '0005', '0006'], dtype=object)

In [1]:
experimentModels[experimentModels.test.isin(np.setdiff1d(experimentModels.test,['0007','0008']))][['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().mean()

NameError: name 'experimentModels' is not defined

In [32]:
def annotationPairToGraph(annotations,detections,thresIoU=0.2):
    #get the coords
    gtCoords=zip(annotations.startInd,annotations.stopInd)
    outCoords=zip(detections.startInd,detections.stopInd)
    #calculate the iou vector
    iouVector=np.array(list(itt.starmap(getIou,itt.product(gtCoords,outCoords))))
    #reshape to a matrix
    iouMatrix=iouVector.reshape(len(annotations),len(detections))
    #create tables
    index0=np.apply_along_axis(np.argmax,0,iouMatrix)
    iou0=np.apply_along_axis(np.max,0,iouMatrix)
    index1=np.apply_along_axis(np.argmax,1,iouMatrix)
    iou1=np.apply_along_axis(np.max,1,iouMatrix)
    
    tableOut=pd.DataFrame({
        'indexGT':index0,
        'iou':iou0
    })

    tableGT=pd.DataFrame({
        'indexOut':index1,
        'iou':iou1
    })

    tableOut['type']='out'
    tableGT['type']='gt'
    tableOut['indexOut']=tableOut.index
    tableGT['indexGT']=tableGT.index
    #Correct external indexes of objects not overlapping
    tableOut.loc[tableOut.iou==0,'indexGT']='NA'
    tableGT.loc[tableGT.iou==0,'indexOut']='NA'
    #set tps
    tableOut['tp']=tableOut.iou>thresIoU
    tableOut['fp']=tableOut.iou<=thresIoU
    tableGT['tp']=tableGT.iou>thresIoU
    tableGT['fn']=tableGT.iou<=thresIoU
    #calculate metrics
    recall=np.sum(tableGT['tp'])/len(tableGT)
    precision=np.sum(tableOut['tp'])/len(tableOut)
    f1=(np.sum(tableGT['tp'])+np.sum(tableOut['tp']))/(len(tableGT)+len(tableOut))
    #concatenate tables
    appended=pd.concat(objs=(tableOut,tableGT),axis=0)
    #modify values
    appended['x']=appended['indexGT']
    appended['y']=appended['indexOut']
    appended.loc[((appended.type=='out')&(~ appended.tp)),'x']=-10
    appended.loc[((appended.type=='gt')&(~ appended.tp)),'y']=-10
    appended['size']=1
    appended.loc[appended.type=='out','size']=3
    #create the graph
    minTPIoU=np.min(appended[appended.tp].iou)
    fig=px.scatter(appended,x='x',y='y',color='iou',symbol='type',
    opacity=0.8,symbol_map={'out':'circle-open','gt':'circle'},size='size',
    color_continuous_scale=
        ((0.0, 'rgb(40,40,40)'),
        (0.000001, 'rgb(28,227,255)'),
        (0.14, 'rgb(56,199,255)'),
        (0.29, 'rgb(85,170,255)'),
        (0.42, 'rgb(113,142,255)'),
        (0.57, 'rgb(142,113,255)'),
        (0.71, 'rgb(170,85,255)'),
        (0.86, 'rgb(199,56,255)'),
        (1.0, 'rgb(227,28,255)')),
    range_x=(-20,len(tableGT)+10),range_y=(-20,len(tableOut)+10),
    title='by-Event evaluation summary<br><sup>F1(@IoU>'+str(thresIoU)+')='+str(round(f1,4))+' | minimum TP IoU: '+str(round(minTPIoU,4))+'</sup>',
    hover_data={'x':False,
    'y':False,
    'tp':False,
    'fp':False,
    'fn':False,
    'size':False,
    'type':False,
    'iou':':.4f', # customize hover for column of y attribute
    'indexGT':True,
    'indexOut':True
    })
    for t in fig.data:
        t.marker.line.width = 2
    fig.update_xaxes(title_text=str(len(tableGT))+' ANNOTATIONS | recall(@IoU>'+str(thresIoU)+')= '+str(round(recall,4)))
    fig.update_yaxes(title_text=str(len(tableOut))+' DETECTIONS | precision(@IoU>'+str(thresIoU)+')= '+str(round(precision,4)))
    fig.add_vline(x=-5,line_dash='dash')
    fig.add_hline(y=-5,line_dash='dash')
    #----------------------------------------------------------------------->
    # https://stackoverflow.com/questions/61827165/plotly-how-to-handle-overlapping-colorbar-and-legends
    # @vestland answer
    """ fig.update_layout(coloraxis_colorbar=dict(yanchor="top", y=1, x=0,
                                            ticks="outside",
                                            ticksuffix=" bills")) """
    # @bitbang answer
    fig.update_layout(legend_orientation="h")
    #<----------------------------------------------------------------------
    return fig

In [33]:
annotationPairToGraph(gtAnnotations,processedAnnotations)